In [61]:
import pandas as pd

In [1]:
import pickle
gun_control = pickle.load(open("gun_control.p", "rb"))

In [6]:
for art in gun_control['articles']:
    art['content']

In [24]:
gun_control['articles'][0]['title']


u"Questions and answers about Obama's executive plan on guns"

In [120]:
from vaderSentiment import vaderSentiment

### take articles in the pickle, match with articles

In [377]:
id_to_title = pd.read_csv("id_to_title_mapping.csv")

In [378]:
id_to_title = id_to_title[(id_to_title['Unnamed: 2'].isnull() == False) & (id_to_title['title'] != '?')]

In [379]:
titles = id_to_title['title'].values

In [227]:
articles = gun_control['articles']
article_titles = []
for art in articles:
    article_titles.append(art['title'].encode("ascii", "ignore"))

In [246]:
not_included = []
title_idx = {}
for title in titles:
    if title in article_titles:
        counter = 0
        for art in article_titles:
            if art == title:
                title_idx[title] = counter
            counter+=1
        #title_idx[title] = np.where(article_titles == title)
    else:
        print title, np.where(titles == title)
        not_included.append(title)

In [380]:
from collections import defaultdict
idx_to_title = {}
for k, v in title_idx.iteritems():
    idx_to_title[v] = k

In [384]:
cluster_idx = {}
for title in title_idx:
    cluster_idx[title_idx[title]] = id_to_title[id_to_title['title'] == title]['Unnamed: 2'].values.astype(int)

In [385]:
cluster_idx

{5: array([0]),
 6: array([0]),
 7: array([4]),
 11: array([0]),
 12: array([5]),
 13: array([3]),
 14: array([3]),
 15: array([5]),
 16: array([0]),
 17: array([0]),
 18: array([0]),
 20: array([3]),
 21: array([3]),
 24: array([3]),
 25: array([4]),
 26: array([1]),
 27: array([1, 3]),
 28: array([1, 1, 1]),
 29: array([0]),
 30: array([0]),
 36: array([8, 4]),
 42: array([4]),
 46: array([0])}

#### frequency counts
- unigrams, bigrams, trigrams

In [44]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
stop = stopwords.words('english')
import string

def tokenize_doc(doc):
    """ 
        basic tokenization
    """
    wordnet_lemmatizer = WordNetLemmatizer()
    punct = string.punctuation
    toks = nltk.word_tokenize(doc)
    toks = [wordnet_lemmatizer.lemmatize(tok) for tok in toks if tok not in string.punctuation]
    return toks


def ngram_count(toks, n):
    grams = {}
    if (n==1):
        #for unigrams take out stop
        ngs = [toks for toks in toks if toks not in stop]
    elif (n==2):
        ngs = nltk.bigrams(toks)
    elif (n==3):
        ngs = nltk.trigrams(toks)
    else:
        print "no ", n, " only up to n=3"
        return None
    fdist = nltk.FreqDist(ngs)
    #decide threshold comparing to toks
    for k,v in fdist.items():
        grams[k] = v
    sorted_grams = sorted(grams.items(), key=lambda x: x[1], reverse=True)[0:15]
    return sorted_grams

In [47]:
doc_tok = tokenize_doc(gun_control['articles'][0]['content'])
print ngram_count(doc_tok, 1), "\n"
print ngram_count(doc_tok, 2), "\n"
print ngram_count(doc_tok, 3), "\n"

[(u'gun', 21), (u'Obama', 15), (u"'s", 11), (u'background', 6), (u'say', 6), (u'check', 6), (u'The', 6), (u'could', 5), (u"n't", 5), (u'new', 5), (u'step', 5), (u'license', 5), (u'ha', 5), (u'Congress', 4), (u'action', 4)] 

[((u'background', u'check'), 6), ((u'in', u'the'), 5), ((u'Obama', u"'s"), 5), ((u'of', u'gun'), 4), ((u'Obama', u'is'), 4), ((u'a', u'license'), 4), ((u'up', u'to'), 4), ((u'San', u'Bernardino'), 3), ((u'on', u'gun'), 3), ((u'you', u"'re"), 3), ((u'gun', u'death'), 2), ((u'gun', u'sale'), 2), ((u'business', u"''"), 2), ((u'gun', u'from'), 2), ((u'or', u'online'), 2)] 

[((u'at', u'gun', u'show'), 2), ((u'the', u'business', u"''"), 2), ((u'There', u"'s", u'no'), 2), ((u'to', u'background', u'check'), 2), ((u'without', u'a', u'license'), 2), ((u"'s", u'up', u'to'), 2), ((u'you', u"'re", u'not'), 2), ((u'business', u"''", u'of'), 2), ((u'San', u'Bernardino', u'California'), 2), ((u"''", u'of', u'selling'), 2), ((u'in', u'the', u'business'), 2), ((u'clarify', u'who', 

#### spacy ngrams

In [15]:
import textacy
from spacy.en import English
import textacy_keyterms as keyterms
parser = English()

In [18]:
import numpy as np

In [92]:
def textacy_sg_rank(doc):
    doc = textacy.TextDoc(doc.strip(), lang = 'en')
    ranked_results = keyterms.sgrank(doc)
    median = np.median([res[1] for res in ranked_results])
    mean = np.mean([res[1] for res in ranked_results])
    return ranked_results, median, mean, doc

In [249]:
def get_article_keyword(text):
    """
        for now pass in article title + content appended together.
    """
    txt_res = textacy_sg_rank(text)
    return txt_res[0], txt_res[1], txt_res[2]

In [281]:
def get_article_text(idx):
    return gun_control['articles'][idx]['title'] + ". " + gun_control['articles'][idx]['content']

### For each article we marked, find ngrams

In [285]:
article_keywords_dict = {}
for art in title_idx:
    temp_dict = {}
    temp_dict['title'] = art
    text = get_article_text(title_idx[art])
    keywords, median, mean = get_article_keyword(text)
    temp_dict['keywords'] = keywords
    temp_dict['median'] = median
    temp_dict['mean'] = mean
    article_keywords_dict[title_idx[art]] = temp_dict  

In [286]:
article_keywords_dict

{5: {'keywords': [(u'President Obama', 0.8309207163630251),
   (u'gun', 0.05494364343204969),
   (u'requirement', 0.04987999743081177),
   (u'violence', 0.034255642774113755),
   (u'focus', 0.030000000000000006)],
  'mean': 0.20000000000000004,
  'median': 0.049879997430811771,
  'title': 'TULSA WORLD-CLASSIFIED ONLY'},
 6: {'keywords': [(u'President Barack Obama \u2019s executive',
    0.16186022093474708),
   (u'licensed dealer', 0.07073005962396912),
   (u'ATF Form', 0.05329185765991512),
   (u'federal guide', 0.04157313035365152),
   (u'gun dealer', 0.041477567190739036),
   (u'Gun show', 0.04124801982306676),
   (u'firearm dealer', 0.03726263540202588),
   (u'private party', 0.035045027399619885),
   (u'background check', 0.03475529572361633),
   (u'Private Party', 0.02737574580992616)],
  'mean': 0.054461955992127699,
  'median': 0.041362793506902897,
  'title': 'Gun-Buying Laws 2016: How To Buy A Firearm Online And At Gun Shows Under New Obama Rules'},
 7: {'keywords': [(u'tragi

### find sentence with the keyword to get sentiment

In [332]:
from textblob import TextBlob

In [417]:
def return_key_sent_dict(doc, keywords):
    keyword_sentence = {}
    for kwd in keywords:
        keyword_sentence[kwd] = []
    for s in doc.sents:
        sentence = s.orth_.lower()
        for kwd in keywords:
            if kwd in sentence:
                temp_dict = {}
                temp_dict["sentiment"] = TextBlob(sentence).sentiment
                temp_dict["sentence"] = s.orth_
                keyword_sentence[kwd].append(temp_dict)
#                 keyword_sentence[kwd] = temp_dict
    return keyword_sentence


In [418]:
article_keyword_sentence_dict = {}
for idx in article_keywords_dict:
    doc = parser(get_article_text(idx))
    keywords = [x[0] for x in article_keywords_dict[idx]['keywords']]
    article_keyword_sentence_dict[idx] = return_key_sent_dict(doc, keywords)

### check

In [ ]:
num = 

In [423]:
article_keyword_sentence_dict[12]

{u'Obama': [],
 u'Paul Ryan': [],
 u'Speaker Paul Ryan': [],
 u'Wednesday': [],
 u'distraction': [{'sentence': u"Paul Ryan: Obama's actions on guns 'a distraction'.",
   'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)},
  {'sentence': u'Read: Paul Ryan\'s 2016 plans start with Obamacare repeal vote \nRyan didn\'t rule out suing the administration to block its executive actions to expand background checks for private guns sales, saying, "we will look at all of our options but we will not take this distraction for more than it is -- a distraction."',
   'sentiment': Sentiment(polarity=0.25, subjectivity=0.4375)}],
 u'executive action': [{'sentence': u'Washington (CNN) House Speaker Paul Ryan ripped President Barack Obama on Wednesday, saying he should be more focused on combating terrorism than issuing new executive actions to address gun sales.',
   'sentiment': Sentiment(polarity=0.3181818181818182, subjectivity=0.4772727272727273)},
  {'sentence': u'Read: Paul Ryan\'s 2016 plan

In [391]:
idx_to_title[12]

"Paul Ryan: Obama's actions on guns 'a distraction'"

In [390]:
cluster_idx[12]

array([5])

In [397]:
cluster_meanings[5]

'against obama (and pro gun)'

### cluster meanings?

In [395]:
cluster_meanings = {0: "neutral",
1: "guns as deadly weapons (anti guns)",
2: "owning guns can be safe and legal (pro gun)",
3: "lack of good gun laws + more background checks for gun owning (could be against or pro)",
4: "Pro obama (in this case saying that his speech was good) ?",
5: "against obama (and pro gun)",
6: "how to keep your guns/ owning a gun is a RIGHT/ pro owning guns and second ammendment e.g. (Gun-Buying Laws 2016: How To Buy A Firearm Online And At Gun Shows Under New Obama Rules)",
7: "Against obama’s speech or there needs to be more done. (anti gun)",
8: "against republicans/against congress (anti gun)"}

In [396]:
cluster_meanings[0]

'neutral'

### now run this on all articles